In [ ]:

# Import necessary libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Data loading and preprocessing
def load_climate_data(file_path):
    # Use scipy engine to handle .gz compressed files
    return xr.open_dataset(file_path, engine='scipy')

def preprocess_data(ds, variables, time_range):
    # Select relevant variables and time range
    data = ds[variables].sel(time=slice(*time_range))

    # Convert to DataFrame for easier handling
    df = data.to_dataframe().reset_index()

    # Handle missing values
    df = df.dropna()

    return df

# Load and preprocess your data
data = load_climate_data('/content/gistemp1200_GHCNv4_ERSSTv5.nc.gz')
processed_data = preprocess_data(data, variables=['tempanomaly'], time_range=('2000-01-01', '2022-12-31'))

# Define the PhysicsInformedLayer
class PhysicsInformedLayer(tf.keras.layers.Layer):
    def __init__(self, units):
        super(PhysicsInformedLayer, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer='random_normal',
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,),
            initializer='zeros',
            trainable=True,
        )

    def call(self, inputs):
        # Apply physical constraints (e.g., conservation of energy)
        constrained_inputs = self.apply_physical_constraints(inputs)
        return tf.matmul(constrained_inputs, self.w) + self.b

    def apply_physical_constraints(self, inputs):
        # Implement physical constraints here
        # For example, ensure energy conservation
        return tf.clip_by_value(inputs, -1, 1)

# Create the model
def create_climate_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = PhysicsInformedLayer(64)(inputs)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1)(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Prepare your data
X = processed_data[['tempanomaly']].values
y = processed_data['tempanomaly'].values  # Predict temperature anomaly based on historical data

# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and compile the model
model = create_climate_model(input_shape=X_train.shape[1:])
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=100, validation_split=0.2, verbose=1)

# Evaluate and make predictions
test_loss = model.evaluate(X_test, y_test)
print(f"Test loss: {test_loss}")
predictions = model.predict(X_test)

# Visualize results
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')